# Clustering Crypto

In [25]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [26]:
# Load the crypto_data.csv dataset.
file_path= "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [27]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1252 entries, 42 to PUNK
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1252 non-null   object 
 1   Algorithm        1252 non-null   object 
 2   IsTrading        1252 non-null   bool   
 3   ProofType        1252 non-null   object 
 4   TotalCoinsMined  744 non-null    float64
 5   TotalCoinSupply  1252 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 59.9+ KB


In [28]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"]==True]
print(crypto_df.shape)
crypto_df.head(10)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [29]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df.dropna(subset= ['Algorithm'] , axis=0, how="any")
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [30]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns= 'IsTrading', axis=1)
print(crypto_df.shape)
crypto_df.head(10)

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [31]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna(axis=0, how= "any")
print (crypto_df.shape)
crypto_df.head(10)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [32]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
display(crypto_df.shape)
crypto_df.head(10)

(532, 5)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [33]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = pd.DataFrame(crypto_df['CoinName'], index= crypto_df.index)
display(crypto_names_df.shape)
crypto_names_df.head()

(532, 1)

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [34]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns= ["CoinName"], axis= 1)
display(crypto_df.shape)
crypto_df.head(10)

(532, 4)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [35]:
# Use get_dummies() to create variables for text features.
x= pd.get_dummies(crypto_df, columns= ['Algorithm', 'ProofType'])
display(x.shape)
x.head(10)

(532, 98)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
x_scaled= scaler.fit_transform(x)
x_scaled[:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [37]:
# Using PCA to reduce dimension to three principal components.

# Initialize PCA Model
pca= PCA(n_components=3)

# Get three principal components for the scaled crypto data x
x_pca = pca.fit_transform(x_scaled)
x_pca

array([[-0.33448808,  0.99796778, -0.57629845],
       [-0.31782462,  0.99804634, -0.57683989],
       [ 2.3033708 ,  1.60812828, -0.63112265],
       ...,
       [ 0.32025269, -2.28944923,  0.36436298],
       [-0.13917415, -1.98045617,  0.45403112],
       [-0.28829969,  0.8501855 , -0.25666661]])

In [38]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data =x_pca, columns= ['PC 1','PC 2','PC 3'], index=crypto_df.index)
display(pcs_df.shape)
pcs_df.head(10)

(532, 3)

,PC 1,PC 2,PC 3
42,-0.334488,0.997968,-0.576298
404,-0.317825,0.998046,-0.576840
1337,2.303371,1.608128,-0.631123
BTC,-0.145495,-1.335760,0.156425
ETH,-0.150866,-2.024770,0.301902
LTC,-0.164807,-1.122070,-0.020334
DASH,-0.401073,1.196750,-0.445034
XMR,-0.148814,-2.185029,0.432795
ETC,-0.149308,-2.024875,0.301871
ZEC,-0.139173,-1.980456,0.454031


### Deliverable 3: Clustering Crytocurrencies Using K-Means
#### Finding the Best Value for k Using the Elbow Curve

In [39]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia=[]
k= list(range(1,11))

# calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Create the elbow curve
elbow_data = {"k":k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y= "inertia", xticks=k, title= "Elbow Curve")

C:\Users\wisam\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with k=4

In [40]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state= 0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [41]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df= crypto_df.merge(pcs_df, left_index=True, right_index=True)
clustered_df.head()
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName']= crypto_names_df['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class']= model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334488,0.997968,-0.576298,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317825,0.998046,-0.576840,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.303371,1.608128,-0.631123,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145495,-1.335760,0.156425,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.150866,-2.024770,0.301902,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.164807,-1.122070,-0.020334,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.401073,1.196750,-0.445034,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.148814,-2.185029,0.432795,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.149308,-2.024875,0.301871,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.139173,-1.980456,0.454031,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results
#### 3D-Scatter with Clusters

In [42]:
# Creating a 3D-Scatter with the PCA data and the clusters
hover = clustered_df["Algorithm"]
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [43]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,Class]

In [44]:
# Print the total number of tradable cryptocurrencies.
clustered_df['TotalCoinsMined'].value_counts


<bound method IndexOpsMixin.value_counts of 42      4.199995e+01
404     1.055185e+09
1337    2.927942e+10
BTC     1.792718e+07
ETH     1.076842e+08
            ...     
ZEPH    2.000000e+09
GAP     1.493105e+07
BDX     9.802226e+08
ZEN     7.296538e+06
XBC     1.283270e+05
Name: TotalCoinsMined, Length: 532, dtype: float64>

In [45]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
C= clustered_df[['TotalCoinsMined','TotalCoinSupply']]
scaler = MinMaxScaler().fit_transform(C)

In [46]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
clustered_df_scaled = pd.DataFrame(data= scaler, columns=['TotalCoinsMined','TotalCoinSupply'])
clustered_df_scaled.index= clustered_df.index

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
clustered_df_scaled['CoinName']=crypto_names_df

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
clustered_df_scaled['Class']= model.labels_

clustered_df_scaled.head(10)

,TotalCoinsMined,TotalCoinSupply,CoinName,Class
42,0.000000,4.200000e-11,42 Coin,0
404,0.001066,5.320000e-04,404Coin,0
1337,0.029576,3.141593e-01,EliteCoin,0
BTC,0.000018,2.100000e-05,Bitcoin,3
ETH,0.000109,0.000000e+00,Ethereum,3
LTC,0.000064,8.400000e-05,Litecoin,3
DASH,0.000009,2.200000e-05,Dash,0
XMR,0.000017,0.000000e+00,Monero,3
ETC,0.000115,2.100000e-04,Ethereum Classic,3
ZEC,0.000007,2.100000e-05,ZCash,3


In [47]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_df_scaled.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by='Class')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)